Actividad 1 

Explicacion del preprocesamiento hecho y/o fotos del preprocesamiento del csv

Actividad 2 

En este caso se usó <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html">KFold</a>. Recuerde que también existe <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split</a>.


In [178]:
from IPython.display import display
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Cargar los datos desde el archivo CSV
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Separar las características y la variable objetivo
X = datos.iloc[:, :-1]  
y = datos.iloc[:, -1]

# Eliminar la columna "Id"
X = X.drop('Id', axis=1)

# Convertir variables categóricas en dummies
X = pd.get_dummies(X)

# Convertir las etiquetas categóricas en valores numéricos
y = pd.get_dummies(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

'''
# Mostrar las primeras filas de los conjuntos de datos
print("Conjunto de entrenamiento:")
display(X_train.head())
display(y_train.head())

print("\nConjunto de prueba:")
display(X_test.head())
display(y_test.head())

# Crear el modelo de red neuronal
'''

model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))  # Capa de entrada
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Capa de salida con dos neuronas para clasificación binaria con función softmax

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f'Pérdida en el conjunto de prueba: {loss:.4f}')
print(f'Precisión en el conjunto de prueba: {accuracy:.4f}')


Pérdida en el conjunto de prueba: 0.6929
Precisión en el conjunto de prueba: 0.5348


Decidimos utilizar la validación cruzada porque nos permite evaluar cómo se comportará nuestro modelo en diferentes conjuntos de datos. Esto nos da una visión más robusta de su desempeño y nos ayuda a evitar sesgos que podrían surgir al dividir los datos en un solo conjunto de entrenamiento y prueba.

Antes de aplicar la validación cruzada, nos aseguramos de que nuestros datos estuvieran limpios y listos para usar. Eliminamos columnas innecesarias, convertimos las categorías en valores numéricos y tratamos los valores faltantes para garantizar la coherencia y la integridad de nuestros datos.

Actividad 3

En este caso debieran que ser 4 bloques 2 modelos y dos modificaciones de parametros para cada modelo 

In [179]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

# Lista para almacenar las precisiones de cada fold
accuracies = []

# Aplicar K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Suponiendo que usamos 5 folds
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear el modelo
    model = DecisionTreeClassifier()

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Hacer predicciones
    y_pred = model.predict(X_test)

    # Agregar predicciones a la lista
    accuracies.append(accuracy_score(y_test, y_pred))
    print(f'Fold accuracy: {accuracy_score(y_test, y_pred):.4f}')

    print(y_pred)

# Mostrar la precisión promedio
print(f'\nAverage accuracy: {np.mean(accuracies):.4f}')

Fold accuracy: 0.6722
[[ True False]
 [False  True]
 [False  True]
 ...
 [False  True]
 [ True False]
 [False  True]]
Fold accuracy: 0.6540
[[ True False]
 [ True False]
 [ True False]
 ...
 [False  True]
 [ True False]
 [False  True]]
Fold accuracy: 0.6556
[[ True False]
 [False  True]
 [ True False]
 ...
 [False  True]
 [False  True]
 [ True False]]
Fold accuracy: 0.6650
[[ True False]
 [ True False]
 [ True False]
 ...
 [False  True]
 [ True False]
 [ True False]]
Fold accuracy: 0.6783
[[ True False]
 [ True False]
 [False  True]
 ...
 [False  True]
 [False  True]
 [ True False]]

Average accuracy: 0.6650


Optamos por probar dos modelos diferentes, Random Forest y Árbol de Decisión. Seleccionamos Random Forest debido a su capacidad para manejar conjuntos de datos grandes y complejos, mientras que elegimos Árbol de Decisión por su simplicidad y facilidad de interpretación.

Creemos que Random Forest es como un equipo diverso que trabaja en colaboración para tomar decisiones, mientras que el Árbol de Decisión es más como un individuo que sigue una serie de reglas predefinidas. Queríamos explorar ambas opciones para obtener una comprensión completa de cómo se comportarán nuestros modelos en nuestro problema específico.

Actividad 4

Aplicacion del modelo a nuevo conjunto

In [180]:
import pandas as pd

import pandas as pd
import numpy as np

# Cargar los datos de evaluación
archivo_evaluacion = 'Taller_2_Titulacion_Evaluación.csv'
datos_evaluacion = pd.read_csv(archivo_evaluacion, encoding='latin-1', delimiter=';')

# Eliminar la columna "Id" si es necesario
X_evaluacion = datos_evaluacion.drop('Id', axis=1)

# Convertir variables categóricas en dummies si es necesario
X_evaluacion = pd.get_dummies(X_evaluacion)

# Asegúrate de que las columnas en X_evaluacion coincidan con las columnas en X_taller
X_evaluacion = X_evaluacion.reindex(columns=X.columns, fill_value=0)

# Hacer predicciones con el mejor modelo encontrado
predicciones_evaluacion = model.predict(X_evaluacion)

# Convertir las predicciones a etiquetas legibles
predicted_labels = np.where(np.argmax(predicciones_evaluacion, axis=1), 'SÍ', 'NO')

# Crear un DataFrame con las columnas Id y Prediccion
resultados = pd.DataFrame({'Id': datos_evaluacion['Id'], 'Prediccion': predicted_labels})

# Guardar el DataFrame como un archivo de texto separado por comas
resultados.to_csv('clasificacion_titulo.txt', index=False, sep=',', header=False)
